In [153]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [154]:
X_full = pd.read_csv(r"D:\Housing price\train.csv", index_col= 'Id')
X_test_full = pd.read_csv(r"D:\Housing price\test.csv", index_col= 'Id')

In [155]:
X_test_full.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [156]:
X_full.shape

(1460, 80)

In [157]:
y = X_full.SalePrice

In [158]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)

In [159]:
#Select categorical columns with relatively low cardinality
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

In [160]:
#select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [161]:
# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [162]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [163]:
my_model = XGBRegressor()
my_model.fit(X_train, y_train)
predictions = my_model.predict(X_valid)
print("MAE of model 0 \n" + str(mean_absolute_error(predictions, y_valid)))

MAE of model 0 
1373.1772126498288


In [164]:
# early_stopping_rounds find value for n_estimators
my_model_1 = XGBRegressor(n_estimators=500)
my_model_1.fit(X_train, y_train, 
               eval_set=[(X_valid, y_valid)],
               verbose=False, 
               early_stopping_rounds=2)
prediction_1 = my_model_1.predict(X_valid)
print("MAE of model early_stopping_rounds \n" + str(mean_absolute_error(prediction_1, y_valid)))

C:\Users\suyog\anaconda3\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


MAE of model early_stopping_rounds 
1348.9973779965753


In [165]:
# learning_rate Instead of getting predictions by simply adding up the predictions from each component model, we can multiply the predictions from each model by a small number (known as the learning rate)
my_model_2 = XGBRegressor(n_estimators=500, learning_rate=0.05)
my_model_2.fit(X_train, y_train)
prediction_2 = my_model_2.predict(X_valid)
print("MAE of model learning_rate \n"+ str(mean_absolute_error(prediction_2, y_valid)))

MAE of model learning_rate 
918.633655286815


In [169]:
# n_jobs On larger datasets where runtime is a consideration, you can use parallelism to build your models faster
my_model_3 = XGBRegressor(n_estimators=500,
                          learning_rate=0.05,
                          n_jobs=10)
my_model_3.fit(X_train, y_train)
prediction_3 = my_model_3.predict(X_valid)
print("MAE of model n_jobs \n"+ str(mean_absolute_error(prediction_3, y_valid)))

MAE of model n_jobs 
918.633655286815
